<a href="https://colab.research.google.com/github/AlexFly666/LLM-in-Practice/blob/main/chapter09/0_quikstart/langchain_semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 构建语义搜索引擎
本教程将深入讲解 LangChain 中用于数据检索的关键概念：文档加载器、嵌入和向量存储。通过本教程，你将学会如何利用这些工具从各种来源（包括向量数据库）检索数据，并将其无缝集成到大型语言模型（LLM）的工作流程中。这些技术是构建强大的 AI 应用，特别是检索增强生成（RAG）应用的基础。掌握这些技能，你将能够开发出能够理解并回答复杂问题的智能应用。

这个代码示例演示了如何使用 LangChain 构建一个基于 PDF 文档的语义搜索引擎。它包含了文档加载、分割、嵌入生成、向量存储和检索等关键步骤。

文档加载: 使用 PyPDFLoader 从 PDF 文件加载文档，每页一个文档。
文档分割: 使用 RecursiveCharacterTextSplitter 将文档分割成更小的块，以便更好地进行语义搜索。
嵌入生成: 使用 OpenAIEmbeddings 将文本块转换为嵌入向量，这些向量捕捉了文本的语义信息。
向量存储: 使用 Chroma 存储嵌入向量，以便进行高效的相似性搜索。
向量检索: 使用向量存储进行相似性搜索，找到与查询最相关的文档块。
检索器: 创建可重用的检索器对象，方便进行批量查询。
解决的问题:

传统的关键词搜索只能找到包含特定关键词的文档，而语义搜索则可以找到含义与查询相关的文档，即使文档中没有包含查询的关键词。这个代码示例展示了如何使用 LangChain 构建一个语义搜索引擎，从而更准确地找到用户感兴趣的信息。

达到的效果:

通过这个代码示例，你可以：

加载和处理 PDF 文档。
将文本转换为嵌入向量，捕捉语义信息。
使用向量存储进行高效的相似性搜索。
构建一个可重用的检索器对象。
这个示例可以帮助你理解 LangChain 的核心概念，并为构建更复杂的自然语言处理应用打下基础。  代码中添加了详细的中文注释，解释了每个步骤的作用和目的，方便理解和学习。



In [ ]:
# !pip install langchain \
#             langchain-community \
#             langchain-openai \
#             langchain-text-splitters \
#             pypdf \
#             langchain_chroma
# LangChain0.3版本
!pip install langchain==0.3.17 \
            langchain-community==0.3.16 \
            langchain-openai==0.3.4 \
            langchain-text-splitters==0.3.5 \
            pypdf==5.2.0 \
            langchain-chroma==0.2.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.5 MB/s eta 0:00:

In [ ]:
!pip show langchain \
            langchain-community \
            langchain-openai \
            langchain-text-splitters \
            pypdf \
            langchain_chroma

Name: langchain
Version: 0.3.17
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: aiohttp, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community
---
Name: langchain-community
Version: 0.3.16
Summary: Community contributed LangChain integrations.
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: aiohttp, dataclasses-json, httpx-sse, langchain, langchain-core, langsmith, numpy, pydantic-settings, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
---
Name: langchain-openai
Version: 0.3.4
Summary: An integration package connecting OpenAI and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-pa

In [ ]:
# 导入必要的库
import getpass
import os
import subprocess
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import chain
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# 设置 LangSmith 追踪，用于调试和监控 LangChain 应用
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

# 1. 加载文档
# 使用 PyPDFLoader 从 PDF 文件加载文档
# 远程 PDF 文件 URL
def download_file(url, file_name):
    try:
        # 使用 curl 下载文件
        subprocess.run(["curl", "-L", url, "-o", file_name], check=True)
        print(f"文件 {file_name} 下载成功")
        return True
    except subprocess.CalledProcessError as e:
        print(f"文件下载失败: {e}")
        return False

# 远程 PDF 文件 URL
raw_url = "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/docs/example_data/nke-10k-2023.pdf"
file_name = "nke-10k-2023.pdf"

# 检查本地文件是否存在，如果不存在则下载
if not os.path.exists(file_name):
    if not download_file(raw_url, file_name):
        print("无法下载 PDF 文件，程序退出")
        exit()

# 使用 PyPDFLoader 加载 PDF 文件
loader = PyPDFLoader(file_name)
docs = loader.load()


print(f"加载了 {len(docs)} 个文档（每页一个文档）")

# 打印第一个文档的内容和元数据
print(f"第一个文档的内容（前200个字符）：\n{docs[0].page_content[:200]}\n")
print(f"第一个文档的元数据：\n{docs[0].metadata}")


# 2. 分割文档
# 使用 RecursiveCharacterTextSplitter 将文档分割成更小的块
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

print(f"将文档分割成 {len(all_splits)} 个块")

# 3. 生成嵌入
# 使用 OpenAIEmbeddings 创建嵌入模型
# embeddings = OpenAIEmbeddings()

# OpenAI 词嵌入（代理方式）
embeddings = OpenAIEmbeddings(
    api_key="XXX",
    base_url="https://vip.apiyi.com/v1"
)



# 示例：生成两个文本块的嵌入向量并比较长度
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)  # 确保向量长度一致
print(f"生成的向量长度为 {len(vector_1)}\n")
print(f"第一个向量的前10个元素：\n{vector_1[:10]}")

# 4. 创建向量存储
# 使用 Chroma 创建向量存储
vector_store = Chroma(embedding_function=embeddings)

# 将分割后的文档块添加到向量存储
# ids = vector_store.add_documents(documents=all_splits)
batch_size = 100  # 设置批次大小
for i in range(0, len(all_splits), batch_size):
    batch = all_splits[i:i + batch_size]
    vector_store.add_documents(documents=batch)

print(f"将 {len(all_splits)} 个文档块添加到向量存储")


# 5. 使用向量存储进行搜索
# 示例：使用相似性搜索查询文档
results = vector_store.similarity_search(
    "How many distribution centers does Nike have in the US?"
)

print(f"相似性搜索结果：\n{results[0]}")

# 示例：异步查询
async def async_search():
    results = await vector_store.asimilarity_search("When was Nike incorporated?")
    print(f"异步搜索结果：\n{results[0]}")

#import asyncio
#asyncio.run(async_search())


# 示例：返回相似性得分
results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")
doc, score = results[0]
print(f"相似性得分：{score}\n")
print(f"带得分的搜索结果：\n{doc}")

# 示例：使用嵌入向量搜索
embedding = embeddings.embed_query("How were Nike's margins impacted in 2023?")
results = vector_store.similarity_search_by_vector(embedding)
print(f"基于向量的搜索结果：\n{results[0]}")


# 6. 创建检索器
# 自定义检索器示例
@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)

# 批量检索示例
retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

# 使用 as_retriever 创建检索器
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

# 批量检索示例
retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)

print("检索器示例完成")

加载了 107 个文档（每页一个文档）
第一个文档的内容（前200个字符）：
Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
F

第一个文档的元数据：
{'source': 'nke-10k-2023.pdf', 'page': 0, 'page_label': '1'}
将文档分割成 516 个块
生成的向量长度为 1536

第一个向量的前10个元素：
[-0.00865495577454567, -0.03345286101102829, -0.00998399406671524, -0.005063311196863651, 0.009030976332724094, 0.009439412504434586, -0.028214506804943085, -0.016467107459902763, 0.0030033020302653313, -0.012771732173860073]
将 516 个文档块添加到向量存储
相似性搜索结果：
page_content='direct to consumer operations sell products through the following number of retail stores in the United States:
U.S. RETAIL STORES NUMBER
NIKE Brand factory stores 213 
NIKE Brand in-line stores (including employee-only stores) 74 
Converse stores (including factory stores) 82 
TOTAL 369 
In the United States, NIKE has eight significant distribution centers. Refer to Item 2.